In [80]:
# Train a basic model on the imported dataset 'data.json'

# Import the required libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from matplotlib import pyplot as plt
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import Adam

# Import the dataset
data = pd.read_json('./data.json')
data2 = pd.read_json('./data2.json')

# Print the first 5 rows of the dataset
data.head()
data2.head()


,rating,params
0,7,"{'a': 6.734716534707012, 'b': 0.20063784522768..."
1,7,"{'a': 24.24513149014455, 'b': 0.22983288090211..."
2,6,"{'a': -4.866820755674933, 'b': 0.2659186585626..."
3,5,"{'a': 9.542271727200948, 'b': 0.24863220592619..."
4,7,"{'a': 9.542271727200948, 'b': 0.24863220592619..."


In [81]:
# Restructure data so that the params are in a single column

# Create the dataframe from the params column and extract the internal dict to a new dataframe
X = pd.DataFrame(data, columns=['params'])
X = pd.DataFrame(X.params.values.tolist(), index=X.index).drop(['timeCreated'], axis=1)
# Remove unwanted columns

y = pd.DataFrame(data, columns=['rating'])

# Import 2nd dataset


X2 = pd.DataFrame(data2, columns=['params'])
X2 = pd.DataFrame(X2.params.values.tolist(), index=X2.index)

y2 = pd.DataFrame(data2, columns=['rating'])

In [82]:
# Shuffle and split the dataset into training and testing sets


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=420)

# Normalise the data
scaler = StandardScaler()

scaler.fit(X_train)

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.fit_transform(X_test)

# Scale the 2nd dataset
X2_scaled = scaler.fit_transform(X2)


In [83]:
# Use a conformal prediction model to predict the ratings of the 2nd dataset

# Import the required libraries
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.neighbors import KNeighborsRegressor


# Create a linear regression model
model = LinearRegression()
knn = KNeighborsRegressor(n_neighbors=5)

# Train the model
model.fit(X_train_scaled, y_train)
knn.fit(X_train_scaled, y_train)

# Predict the ratings of the 2nd dataset
y2_pred = model.predict(X2_scaled)
y2_pred_knn = knn.predict(X2_scaled)

# Calculate the mean squared error
mse = mean_squared_error(y2, y2_pred)
mse_knn = mean_squared_error(y2, y2_pred_knn)

# Calculate the r-squared score
r2 = r2_score(y2, y2_pred)
r2_knn = r2_score(y2, y2_pred_knn)

# Print the results
print('Mean squared error: ', mse)
print('R-squared score: ', r2)
print('Mean squared error (knn): ', mse_knn)
print('R-squared score (knn): ', r2_knn)


Mean squared error:  9.911672582143039
R-squared score:  -0.14587644980261216
Mean squared error (knn):  9.746315789473686
R-squared score (knn):  -0.12675975148914365


In [84]:
# Generating the best frames for the orbit visualizer

# Randomly generate 1000 frames, get the model to classify them and save the best 100 frames

import random

# Generate 10000 random frames within the ranges of the various parameters
frames = []
for i in range(1000000):
		frame = []
		frame.append(random.uniform(-30, 30)) # a
		frame.append(random.uniform(0.2, 1.8)) # b
		frame.append(random.uniform(5, 17)) # c
		frame.append(random.uniform(0, 10)) # d
		frame.append(random.uniform(0, 12)) # e
		frame.append(random.uniform(0, 1)) # choice
		frame.append(random.uniform(0, 1)) # xPreset
		frame.append(random.uniform(0, 1)) # yPreset
		frames.append(frame)

# Get the model to classify the frames
frames_scaled = scaler.fit_transform(np.array(frames))

# Get the model to predict the rating for the frames
frames_pred = np.add(model.predict(frames_scaled), knn.predict(frames_scaled))

# Map the frames to their predicted ratings
frames_pred = np.concatenate((frames, frames_pred), axis=1)

# Sort the frames by their predicted ratings
frames_pred = frames_pred[frames_pred[:,8].argsort()]

# Save the best 100 frames
best_frames = frames_pred[-1000:]
print(best_frames)



[[-2.74357924e+01  2.21519553e-01  1.39293102e+01  2.51391980e-01
   1.06244835e+01  9.33649574e-01  4.62936679e-01  3.22621923e-01
   1.76470013e+01]
 [-2.14747670e+01  2.29667336e-01  6.17398961e+00  2.46950899e+00
   6.49666201e-01  9.21525990e-01  8.81471291e-01  4.32497310e-01
   1.76482704e+01]
 [-2.09749495e+01  2.59032084e-01  1.08500863e+01  3.74654319e+00
   6.84895299e+00  9.43324422e-01  9.50207137e-01  3.16030390e-01
   1.76495247e+01]
 [-1.95670957e+01  2.00787121e-01  1.43493609e+01  4.14496933e+00
   7.68005253e+00  9.13860729e-01  8.28166289e-01  2.96604365e-01
   1.76515173e+01]
 [-1.60491580e+01  4.70013505e-01  1.40365884e+01  4.00769936e+00
   3.92554165e-01  9.69744482e-01  7.35226079e-01  6.93312968e-01
   1.76542875e+01]
 [-2.99481900e+01  3.63263391e-01  1.66449403e+01  3.59415211e+00
   7.58396745e+00  9.73783799e-01  9.51070011e-01  1.27857625e-01
   1.76630932e+01]
 [-1.70772803e+01  5.02485485e-01  1.49717482e+01  2.23703429e+00
   4.93144118e+00  9.2340998

In [85]:
# Map the best frames to the parameters
mapped = []
for frame in best_frames:
		mapped.append({"rating": frame[8], "params": {
				"a": frame[0],
				"b": frame[1],
				"c": frame[2],
				"d": frame[3],
				"e": frame[4],
				"choice": frame[5],
				"xPreset": frame[6],
				"yPreset": frame[7],
				
		}})

# Convert the best frames to JSON and save them to a file
import json

with open('best_frames.json', 'w') as outfile:
		json.dump(mapped, outfile)